In [1]:
!pip install mysql-connector-python

     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
     ---- -------------------------------- 61.4/527.0 kB 812.7 kB/s eta 0:00:01
     ------------------ ------------------- 256.0/527.0 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  522.2/527.0 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   - -------------------------------------- 0.4/14.2 MB 12.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/14.2 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.3/14.2 MB 10.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/14.2 MB 10.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.5/14.2 MB 11.4 MB/s eta 0:00:02
 

In [6]:
import mysql.connector
from mysql.connector import Error

def execute_sql_file(filename, host, database, user, password):
    # 데이터베이스 연결
    try:
        connection = mysql.connector.connect(host=host,
                                             database=database,
                                             user=user,
                                             password=password)
        if connection.is_connected():
            cursor = connection.cursor()

            # SQL 파일 읽기
            with open(filename, 'r') as file:
                sql_script = file.read()

            # SQL 명령문 실행
            for statement in sql_script.split(';'):
                if statement.strip():
                    cursor.execute(statement)

            connection.commit()
            print(f"SQL script '{filename}' executed successfully")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# 사용 예
execute_sql_file("../backup/db/study_20231228_172556.sql", "localhost", "study_test", "root", "0000")


SQL script '../backup/db/study_20231228_172556.sql' executed successfully
MySQL connection is closed
